In [ ]:
param_dict_copy=param_dict
seasonal_dict_copy=seasonal_dict
result_dict_copy=result_dict

In [ ]:
get_ipython().run_cell_magic('capture', 'cont', '!pip install db-dtypes\n')

In [ ]:
cont.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

In [ ]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [ ]:
import statsmodels.api as sm
import itertools

CONFIGURE THE BIGQUERY SETTINGS

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
#obtaining the unemployment data
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()
#data['date']= pd.to_datetime(data['date'])
data.head()

In [ ]:
#obtaining the submission file
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
submit = query_job.to_dataframe()
#data['date']= pd.to_datetime(data['date'])
submit.head()

In [ ]:
data.info()   #Feature Matrix
#X = data.drop("date",1)
#y = data["wind_speed"]          #Target Variable

In [ ]:
submit.info()

dropping duplicate rows in the data

In [ ]:
data.drop_duplicates(inplace=True, ignore_index=True)
data.info()

unique number of ID in the data

In [ ]:
data.uu_id.unique().size

replace all na values with 0

In [ ]:
data.fillna(0, inplace=True)

In [ ]:
data.info()

number of weeks in the data for a random ID

In [ ]:
test=data[data['uu_id']=='bbcb018f0e5e49e13636f6e78ce9f60f']
len(test)

convert the timeperiod column to datetime

In [ ]:
data['timeperiod']= pd.to_datetime(data['timeperiod'], format='%Y%m%d')

In [ ]:
plt.figure(figsize=(12,10))
cor = data.corr()
sns.heatmap(cor,  cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["total_claims"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.2]
print(relevant_features)

In [ ]:
columns_rel = relevant_features.index.to_list()

In [ ]:
data.plot(subplots=True, figsize=(20,24))

In [ ]:
min(data['timeperiod']),max(data['timeperiod'])

In [ ]:
data.dtypes

data = data.set_index('timeperiod')

In [ ]:
data.index

data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day
Display a random sampling of 5 rows
data.sample(5, random_state=0)

In [ ]:
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
data['total_claims'].plot(linewidth=.5);

In [ ]:
cols_plot = ['week_number','countyfips','tract','total_claims','edu_8th_or_less','edu_grades_9_11','edu_hs_grad_equiv','edu_post_hs',
            'edu_unknown', 'gender_female', 'gender_male', 'gender_na', 'race_amerindian', 'race_asian', 'race_black', 'race_noanswer', 
            'race_hawaiiannative', 'race_other', 'race_white']
axes = data[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(30, 25), subplots=True)
for ax in axes:
    ax.set_ylabel('features')

import matplotlib.dates as mdates
fig, ax = plt.subplots()
ax.plot(data.loc['2019-08':'2019-12', 'precipitation_data'], marker='o', linestyle='-')
ax.set_ylabel('Precipitation')
ax.set_title('Aug 2019-2020 Precipiation Data')

In [ ]:
#fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
#for name, ax in zip(['precipitation_data', 'min_temperature', 'max_temperature'], axes):
#    sns.boxplot(data=data, x='Month', y=name, ax=ax)
#    ax.set_ylabel('precipitation')
#    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
#    if ax != axes[-1]:
#        ax.set_xlabel('')

sns.boxplot(data=data, x='Month', y='total_claims');

from statsmodels.graphics.tsaplots import plot_acf
plot_acf(x=data['max_temperature'], lags=50)

In [ ]:
columns_rel.append('timeperiod')
columns_rel.append('uu_id')
columns_rel

In [ ]:
data

In [ ]:
data_arima=data[columns_rel]
data_arima

In [ ]:
get_ipython().run_cell_magic('capture', '', "unique_id=list(data_arima['uu_id'].unique())\ndata_dict = {}\n\nfor i in unique_id:\n    j = data_arima[data_arima['uu_id']==i].groupby('timeperiod')['total_claims'].sum().reset_index()\n    j = j.set_index('timeperiod')\n    data_dict[i] = j\n")

data_arima = data_arima.groupby('uu_id')[['total_claims', 'timeperiod','uu_id']]
data_arima.head()

data_arima_2 = data_arima.set_index('timeperiod')
data_arima.index

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,0,1),
                                seasonal_order=(1,1,1,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,1,1),
                                seasonal_order=(1,1,1,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,1,1),
                                seasonal_order=(1,1,0,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,0,1),
                                seasonal_order=(1,1,0,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,0,1),
                                seasonal_order=(0,1,0,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,0,1),
                                seasonal_order=(0,1,1,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,1,0),
                                seasonal_order=(1,1,1,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,0,0),
                                seasonal_order=(1,1,1,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
result_dict = {}
param_dict = {}
seasonal_dict = {}
for key in data_dict:
    y=data_dict[key]['total_claims']
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            
            try:
                
                mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)
                result_dict[key] = mod.fit(disp=False)
                #print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, result_dict[key].aic))
                param_dict[key]=param
                seasonal_dict[key]=param_seasonal
            except:
                continue

In [ ]:
param_dict_copy=param_dict
seasonal_dict_copy=seasonal_dict
result_dict_copy=result_dict

In [ ]:
len(result_dict)

In [ ]:
param_dict_copy.to_csv(index=False)

In [ ]:
pd.DataFrame(param_dict_copy).to_csv(index=False)

In [ ]:
pd.DataFrame(param_dict_copy).to_csv('./param.csv',index=False)

In [ ]:
pd.DataFrame(param_dict_copy).to_csv('./param.csv',index=False)
pd.DataFrame(seasonal_dict_copy).to_csv('./seasonal.csv',index=False)

In [ ]:
data_dict.key()

In [ ]:
data_dict.keys()

In [ ]:
data_dict.keys()[:100]

In [ ]:
list(data_dict.keys())

In [ ]:
list(data_dict.keys())[:100]

In [ ]:
list(data_dict.keys())[:10]

In [ ]:
list(data_dict.keys())[:100]

In [ ]:
hundred=list(data_dict.keys())[:100]

In [ ]:
results = {}
for key in hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=(1,1,1),
                                seasonal_order=(1,1,1,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results = {}
for key in hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=param_dict[key],
                                seasonal_order=seasonal_dict[key],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
results

In [ ]:
hundred=list(data_dict.keys())[:100]
two_hundred=list(data_dict.keys())[100:200]

In [ ]:
#results = {}
for key in two_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=param_dict[key],
                                seasonal_order=seasonal_dict[key],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
hundred=list(data_dict.keys())[:100]
two_hundred=list(data_dict.keys())[100:200]
three_hundred=list(data_dict.keys())[200:300]

In [ ]:
#results = {}
for key in three_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=param_dict[key],
                                seasonal_order=seasonal_dict[key],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(results)

In [ ]:
hundred=list(data_dict.keys())[:100]
two_hundred=list(data_dict.keys())[100:200]
three_hundred=list(data_dict.keys())[200:300]
four_hundred=list(data_dict.keys())[300:400]

In [ ]:
#results = {}
for key in four_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=param_dict[key],
                                seasonal_order=seasonal_dict[key],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])

In [ ]:
len(list(data_dict.keys())[500:])

In [ ]:
hundred=list(data_dict.keys())[:100]
two_hundred=list(data_dict.keys())[100:200]
three_hundred=list(data_dict.keys())[200:300]
four_hundred=list(data_dict.keys())[300:400]
five_hundred=list(data_dict.keys())[500:]

In [ ]:
#results = {}
for key in five_hundred:
    y=data_dict[key]['total_claims']
    mod = sm.tsa.statespace.SARIMAX(y.astype(float),
                                order=param_dict[key],
                                seasonal_order=seasonal_dict[key],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results[key] = mod.fit(disp=False)
    #print(results.summary().tables[1])